<a href="https://colab.research.google.com/github/KONEONE/nlp-demo/blob/main/llamaindex_load_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLAMAINDEX 加载 文档
这篇笔记是记录llamaidnex加载文档的学习笔记  
## 加载  
* SimpleDirectoryReader： 用于本地目录加载各种文件类型的加载  
* LlamaParse: LlammaIndex官方的PDF解析工具，作为托管API提供
* LlamaHub: 包含了数百个数据加载库的注册中心  
## 转换  
* 节点解析器：  
* 节点解析器模块：
## 整合所有内容  
设置一个可重复、缓存优化的加载数据的过程
## 高级
文档和节点对象以及高级使用

In [ ]:
# 安装依赖
%pip install llama-index-core
%pip install llama-index-llms-dashscope
%pip install llama-index-indices-managed-dashscope
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file
%pip install llama-index-vector-stores-qdrant
%pip install qdrant-client
%pip install llama-index-storage-kvstore-redis
%pip install redis

In [ ]:
!conda install llama-index-node-parser-dashscope

In [ ]:
!pip install llama-index-embeddings-dashscope

In [6]:
import os

os.environ["DASHSCOPE_API_KEY"] = 'sk-a49fcd63a06f4b669c38b1461a60d2e4'

In [8]:
from llama_index.core import Settings
from llama_index.llms.dashscope import DashScope
from llama_index.embeddings.dashscope import DashScopeEmbedding

# 将LlamaIndex的embeding model 设置为百炼
Settings.embed_model = DashScopeEmbedding(model_name="text-embedding-v1", api_key=os.getenv("DASHSCOPE_API_KEY"))

# 将LlamaIndex的llm model 设置为百炼
Settings.llm = DashScope(model_name="qwen-plus", api_key=os.getenv("DASHSCOPE_API_KEY"))

## 文档&节点 - 概述

In [11]:
### 根据文档构建索引
from llama_index.core import Document, VectorStoreIndex

test_list = ["风急天高猿啸哀", "渚清沙白鸟飞回", "无边落木萧萧下", "不尽长江滚滚来"]
documents = [Document(text=t) for t in test_list]

# 构建index
indexs = VectorStoreIndex.from_documents(documents=documents)

In [ ]:
### 根据Node构建索引
from llama_index.core.node_parser import SentenceSplitter

# doc -段落切割-> node
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# 构建node_index
node_index = VectorStoreIndex(nodes)

### 文档使用  

In [13]:
### 加载文件生成doc
from llama_index.core import SimpleDirectoryReader

base_dir = "./data/mkData"
baseDocuments = SimpleDirectoryReader(base_dir).load_data()

for d in baseDocuments:
  print(f'{d.id_} - {d.get_type()}')

e754b69e-6308-4db2-ab87-15f27567d8fe - 4
02fa5b3e-5e43-4e16-83e0-7193606ac8ec - 4
19190085-cd83-4810-8ec6-418b95134c46 - 4
e161aeb0-48dc-471f-bb4e-7ea048f48e01 - 4
fb22b79b-7484-4a95-b9a2-0df092c95d23 - 4


In [ ]:
### 手动生成doc
from llama_index.core import Document

test_list = ["风急天高猿啸哀", "渚清沙白鸟飞回", "无边落木萧萧下", "不尽长江滚滚来"]
testDocuments = [Document(text=t) for t in test_list]

for d in testDocuments:
  print(f'{d.id_} - {d.get_type()}')

27e89aa5-7057-427f-9e48-3d90fd5418bb - ObjectType.DOCUMENT
cb671995-1bd6-461a-b5f9-6253ad325929 - ObjectType.DOCUMENT
d40ddca0-dc4f-486c-baa8-7ef396133354 - ObjectType.DOCUMENT
9a5e9433-6836-4f71-8a98-ef4efd4e6c7d - ObjectType.DOCUMENT


#### 自定义文档  
任何在文档的metadata字典中设置的信息都会显示在该文档创建的每个源节点metadata中。这些信息包含在节点中，使索引能够在查询和响应中利用它。默认情况下，元数据会注入文本中，用于嵌入和LLM模型调用  


In [14]:
for d in baseDocuments:
  print(f'{d.metadata}')

{'file_path': '/Volumes/Data/code/python_code/nlp-demo/data/mkData/gmall项目-DIM层.md', 'file_name': 'gmall项目-DIM层.md', 'file_size': 7, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}
{'file_path': '/Volumes/Data/code/python_code/nlp-demo/data/mkData/gmall项目-DWD层.md', 'file_name': 'gmall项目-DWD层.md', 'file_size': 9068, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-17'}
{'file_path': '/Volumes/Data/code/python_code/nlp-demo/data/mkData/gmall项目-DWS层.md', 'file_name': 'gmall项目-DWS层.md', 'file_size': 7, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}
{'file_path': '/Volumes/Data/code/python_code/nlp-demo/data/mkData/gmall项目-ODS层.md', 'file_name': 'gmall项目-ODS层.md', 'file_size': 6734, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-06'}
{'file_path': '/Volumes/Data/code/python_code/nlp-demo/data/mkData/图布局算法.md', 'file_name': '图布局算法.md', 'file_size': 2367, 'creation_date': '2024-12-30', 'last_modified_date': '2024-12-30'}


### 使用Node  
在llamaindex中构建节点关系（relationships）是构建结构化知识图谱的关键机制，它使得节点之间形成有意义的连接，这些关系对如下场景非常重要：  
1. 保持文档结构  
1.1 当原始文档被拆分为多个节点时，关系可以重建原始顺序  
1.2 Next/Previous关系维护文本的连续性，就像链表一样链接节点  
2. 增强索引质量  
2.1 索引时可以获取相关节点（如前\后文）提供更加完整的上下文    
2.2 示例：回答问题时，获取匹配节点的前后文能提高答案准确性  
3. 构建知识图谱  
3.1 表示概念间的语义关系（如父子、引用等）  
3.2 PARENT、CHILD关系可以构建层次结构  
4. 图遍历能力  
4.1 支持复杂查询  
4.2 实现多跳推理  


In [ ]:
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

# 创建2个连续文本带你
node1 = TextNode(text="段落1", id="node01")
node2 = TextNode(text="段落2", id="node02")

# 创建双向连接关系
node1.relationships[NodeRelationship.NEXT] = RelatedNodeInfo(node_id="node02")  # node1 下一个节点：node2
node2.relationships[NodeRelationship.PREVIOUS] = RelatedNodeInfo(node_id="node01") # node2 下一个节点： node1

下面给出了NodeRelationship的一些常用关系：

|关系类型 |	说明	| 典型应用|  
|----|----|----|  
|NEXT	|下一个节点|	连续文本|
|PREVIOUS	|前一个节点	|连续文本|
|PARENT	|父节点	|章节结构|
|CHILD	|子节点	|章节结构|
|SOURCE	|源文档	|节点溯源|
|REFERENCE	|引用节点	|跨文档链接|
|CONTEXT	|相关上下文	|补充信息|


### 元数据提取  
元数据模块包含以下“特征提取器”：  
* SummaryExtractor : 自动从一组节点中提取摘要  
* QuestionsAnsweredExtractor: 提取每个接待您可以回答的一组问题  
* TitleExtractor: 从每个节点的上下文提取标题
* EntityExtractor: 提取每个节点内容中提到的实体（地点、人物、事务等）  

In [ ]:
# 导入元数据提取器和节点解析器
from llama_index.core.extractors import (
    TitleExtractor, QuestionsAnsweredExtractor
)
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline

# 1. 创建文本分割器（节点解析器）
text_splitter = TokenTextSplitter(
    separator=" ",      # 使用空格作为分隔符
    chunk_size=512,     # 每个节点的最大token数量
    chunk_overlap=128   # 节点间重叠token数量
)

# 2. 创建元数据提取器
#   2.1 自动生成节点内容的标题
title_extractor = TitleExtractor(
    nodes = 5  # 使用上下5个节点生成上下文标题
)
#   2.2 提取节点能回答的问题
qa_extractor = QuestionsAnsweredExtractor(
    questions=3  # 为每个节点生成能回答的3个问题
)

# 3. 创建数据处理管道
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,    # 步骤1：分割文档
        title_extractor,  # 步骤2：提取标题
        qa_extractor      # 步骤3：生成问题
    ]
)

# 4. 运行管道处理文档
#   4.1 第一种写法
pipelineNodes = pipeline.run(
    documents=baseDocuments,  # 输入docs
    in_place=True,        # 直接修改文档对象
    show_progress= True,  # 显示进度条
)

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 110/110 [04:38<00:00,  2.53s/it]


In [ ]:
for n in pipelineNodes[:4]:
  print(f'{n.id_} -> {n.metadata["document_title"].split("**Title:")[1].replace("**", "")}')

d6bbf91f-433f-4970-9847-2e06e730848b ->  数据库设计与SQL优化实战：从范式理论到AWS IAM跨账号安全访问  

这个标题既涵盖了数据库设计与SQL查询优化的核心内容，也体现了AWS IAM跨账号访问的实际应用场景，体现了文档的技术深度与广度。
18e9063c-2fae-4c84-b78a-a0b2e678eb61 ->  数据库设计与SQL优化实战：从范式理论到AWS IAM跨账号安全访问  

这个标题既涵盖了数据库设计与SQL查询优化的核心内容，也体现了AWS IAM跨账号访问的实际应用场景，体现了文档的技术深度与广度。
8e08e254-0c38-45c0-837c-7a4dd46c92f2 ->  数据库设计与SQL优化实战：从范式理论到AWS IAM跨账号安全访问  

这个标题既涵盖了数据库设计与SQL查询优化的核心内容，也体现了AWS IAM跨账号访问的实际应用场景，体现了文档的技术深度与广度。
939ed30a-9de3-496c-91a3-34cd14eae3e0 ->  数据库设计与SQL优化实战：从范式理论到AWS IAM跨账号安全访问  

这个标题既涵盖了数据库设计与SQL查询优化的核心内容，也体现了AWS IAM跨账号访问的实际应用场景，体现了文档的技术深度与广度。


In [ ]:
#   4.2 第二种写法
from llama_index.core import VectorStoreIndex
nodes_index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter, title_extractor, qa_extractor]
)

## 节点解析器、文本分割器


In [ ]:
# 单独使用
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(
    chunk_size=1024,    # 块大小
    chunk_overlap=256,  # 上下块覆盖大小
)

nodes = node_parser.get_nodes_from_documents(
    documents=baseDocuments,    # 导入doc
    show_progress=True,         # 显示处理进度
)

In [ ]:
# 转换使用
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

# 定义pipeline的操作序列
pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

# 执行nodes
nodes = pipeline.run(documents=baseDocuments)

### 索引使用   
设置在transformations或全局设置中，以便在通过 .from_documents() 构建索引时自动使用

# 数据管道
输入数据中，这些Transformations会被应用到输入的数据中，最终生成node。这些node要么被返回要么被插入到向量数据库中。     
下面是简单的使用：

In [ ]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# 创建pipeline转化
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=0),   # 定义划分node
        TitleExtractor(),     # 给出 title
        OpenAIEmbedding(),    # 创建 embedding
    ]
)

# 生成node
tmpNode = pipeline.run(documents=baseDocuments)

连接到向量数据库  
当输入数据被转换后，将embedding的node插入到远程向量数据库中。  

In [ ]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client

# 创建Qdrant向量数据库
client = qdrant_client.QdrantClient(location=":memory:")
# 创建Qdrant向量存储实例
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

# 创建数据处理管道
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=256),   # 文本分割
        TitleExtractor(),       # 提取块的标题
        OpenAIEmbedding(),      # 生成embedding
    ],
    vector_store=vector_store   # 指定存储的位置
)

# 对数据进行处理
pipeline.run(documents=baseDocuments)

# 从向量数据库中创建index
from llama_index.core import VectorStoreIndex

# 直接从数据库中创建索引
index = VectorStoreIndex.from_vector_store(vector_store)

### 管道缓存  
在IngestionPipeline中，可以缓存pipeline,这样可以节省后续使用相同数据转换的操作

In [ ]:
# 存储pipeline
pipeline.persist(f"./pipeline_storage")
# 加载pipeline
new_pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=256),   # 文本分割
        TitleExtractor(),       # 提取块的标题
        OpenAIEmbedding(),      # 生成embedding
    ],
    vector_store=vector_store   # 指定存储的位置
)
new_pipeline.load("./pipeline_storage")

下面将pipeline数据存储到redis中

In [ ]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache

# 实例化 redis Cache
ingest_cache = IngestionCache(
    cache = RedisCache.from_host_and_port(host="localhost", port=6379),
    collection = "my_test_cache"
)

# 存储到redis中
pipeline = IngestionPipeline(
    transformations=[
      SentenceSplitter(chunk_size=25, chunk_overlap=0),
      TitleExtractor(),
      OpenAIEmbedding(),
    ],
    cache=ingest_cache,
)

redis_cache_nodes = pipeline.run(documents=baseDocuments)

# 使用属性图索引
属性图是一种知识集合，由带标签的节点（即实体类别、文本标签等）组成，这些节点具有属性（即元数据），并通过关系连接成结构化的路径。  
### SimpleLLMPathExtractor 三元组提取器
SimpleLLMPathExtractor是知识三元组提取器，核心就是从非结构化的文本中，自动抽取结构化的知识。并且这些知识以“主语，谓语，宾语”的形式  
主要功能如下：  
* 自动化知识提取  
* 结构化输出  
* 高度可定制化  
* 集成于LlamaIndex工作流  

In [17]:
from llama_index.core.indices.property_graph import SimpleLLMPathExtractor
# 定义一个提示词模板，用于指导LLLM如何从文本中提取知识三元组
prompt = (
    "Some text is provided below. Given the text, extract to"
    "{max_path_per_chunk}"  # 这是一个占位符，会被实际数值替换
    "knowledge triples in the form of `subject,predicate,object` on each line. Avoid stopwords.\n"
    # 提示词要求LLM以“注意，谓词，客体”的格式提取知识三元组，每行一个，并且避免使用停用词
)

# 定义一个解析函数，用于处理LLM响应并将其转化为三元组
def parse_fn(response_str:str) -> list[(str, str, str)]:
    # 将相应字段按照换行符分割为多行
    lines = response_str.split("\n")
    # 对每行按照逗号，分割为三元组
    triples = [line.split(",") for line in lines]
    return triples


# 创建知识图谱提取器实例
kg_extractor = SimpleLLMPathExtractor(
    extract_prompt=prompt,
    parse_fn=parse_fn
)


### ImplicitPathExtractor
这个提取器不需要 LLM 或嵌入模型来运行，因为它只是解析 llama-index 节点对象上已存在的属性 

In [18]:
from llama_index.core.indices.property_graph import ImplicitPathExtractor

kg_imp_extractor = ImplicitPathExtractor()

In [20]:
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor

kg_dm_extractor = DynamicLLMPathExtractor(
    max_triplets_per_chunk=20,      # 限制每一段文本块中最多提取20个三元组，这有利于控制输出数量和质量
    num_workers=4,
    # 【核心功能：实体类型过滤】
    #   指定允许提取的实体类型（主语和宾语的类型）
    #   这是一个强大的过滤机制，提取器会优先识别和提取属于这些类型的实体
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY"],
    # 【核心功能：关系类型过滤】
    #   指定允许提取的关系类型（谓词种类）
    #   提取器只关注并提取这些特定的关系
    allowed_relation_types=["PRESIDENT_OF", "MEMBER_OF"],
)


In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor


# 定义知识图谱中所有的节点类型
entities = Literal['PERSON', 'PLACE', 'THING']
# 定义知识图谱中所有的边类型
relations = Literal['PART_OF', 'HAS', 'IS_A']

# 定义一个字典，精确定义了哪些类型的实体之间可以存在哪些类型的关系
# 这是核心配置，定义了知识图谱的结构
schema = {
    # key: 实体类型
    # value: 该类型实体可以作为主语拥有的关系类型列表
    "PERSON": ["PART_OF", "HAS", "IS_A"], # 含义：人(PERSON)可以是yyy的一部分(PART_OF),可以拥有(HAS)xxx,可以是hhh的一种(IS_A)
    "PLACE": ["PART_OF", "HAS"], # 地方(PLACE)可以是xxx的一部分(PART_OF),可以拥有(HAS)xxx
    "THING": ["IS_A"], # 东西(THING)可以是yyy的一种(IS_A)
}

kg_sc_extractor = SchemaLLMPathExtractor(
    llm=Settings.llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=schema,
    # 【核心参数：严格模式】
    #   1. 如果设置True，提取器只保留符合kg_validation_schema的三元组
    #   2. 如果设置False，不符合kg_validation_schema的也会被保留，但是符合kg_validation_schema的会被优先提取
    strict=True,
    num_workers=4,
    max_triplets_per_chunk=10,
)



### 案例
下面是一个graph的简单案例

In [ ]:
# TODO 1. 索引
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.dashscope import DashScopeEmbedding

graph_index = PropertyGraphIndex.from_documents(
    baseDocuments,
    llm = s
)